In [3]:
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle


In [4]:
cd

/Users/valencialie


In [5]:
cd Desktop/CZ1016_DS2/kdrama/Images/Male

/Users/valencialie/Desktop/CZ1016_DS2/kdrama/Images/Male


In [6]:
path = r"/Users/valencialie/Desktop/CZ1016_DS2/kdrama/Images/Male"
# change the working directory to the path where the images are located
os.chdir(path)

In [7]:
actors = [] #create an empty list to store all the image files

In [8]:
# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.png'):
          # adds only the image files to the flowers list
            actors.append(file.name)

In [9]:
len(actors)

576

In [10]:
cd

/Users/valencialie


In [11]:
cd Downloads/Images/Male

/Users/valencialie/Downloads/Images/Male


In [12]:
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels 1 cause grayscale)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

In [13]:
data = {}

# loop through each image in the dataset
for actor in actors:
    # try to extract the features and update the dictionary
    try:
        feat = extract_features(actor,model)
        data[actor] = feat
    except:
        pass
          
# get a list of the filenames
filenames = np.array(list(data.keys()))

# get a list of just the features
feat = np.array(list(data.values()))
feat.shape

feat = feat.reshape(-1,4096)
feat.shape

(576, 4096)

In [14]:
pca = PCA(n_components=100, random_state=22)
pca.fit(feat)
x = pca.transform(feat)



In [15]:
len(x)

576

In [16]:
array_list = list(x)


In [17]:
array_label = list(data.keys())
d = { label: value for label, value in zip(array_label, array_list) }

In [18]:
df = pd.DataFrame(d.items(), columns=['Actor', 'Array'])

In [19]:
dist_list = []
dist_list2 = []
index = []
zipped_lists = []
sorted_zipped_lists = []
top5 = []
overall = []

In [20]:
for j in range(0,576):
    for i in range(0,576):
        if i == j:
            continue
        else:
            dist = np.linalg.norm(x[j]-x[i])
            dist_list.append(dist)
            dist_list2.append(dist)
            index.append(i)
            dist_list.sort()
            top5 = dist_list[:5]
            zipped_lists = zip(dist_list2, index)
            sorted_zipped_lists = sorted(zipped_lists)
            sorted_list1 = [element for _, element in sorted_zipped_lists]
            top5.extend(sorted_list1[:5])
    overall.append(top5)
    dist_list = []
    dist_list2 = []
    index = []
    zipped_lists = []
    sorted_list1 =[]
    top5 = []

In [21]:
# overall 

In [22]:
df1 = pd.DataFrame(overall,columns=['Least_Array_Diff', '2ndLeast_Array_Diff','3rdLeast_Array_Diff', '4thLeast_Array_Diff', '5thLeast_Array_Diff', '1stMatch', '2ndMatch', '3rdMatch', '4thMatch', '5thMatch'])

In [23]:
df2 = pd.concat([df, df1], axis = 1)

In [24]:
df2.head()

,Actor,Array,Least_Array_Diff,2ndLeast_Array_Diff,3rdLeast_Array_Diff,4thLeast_Array_Diff,5thLeast_Array_Diff,1stMatch,2ndMatch,3rdMatch,4thMatch,5thMatch
0,Park_Hae_Soo.png,"[-4.117684, -9.592239, -7.871775, 1.1058365, 5...",30.715021,30.962044,33.221836,35.256233,35.511070,73,399,175,488,406
1,Lee_Jin_Wook.png,"[65.89964, 15.4892235, -10.712205, -14.894704,...",39.949093,44.084579,47.698715,48.780590,49.159027,519,390,365,141,37
2,Kang_Min_Hyuk.png,"[-12.266893, -7.2090816, 1.4790905, 9.8934355,...",26.389797,30.042631,30.068354,31.712767,31.934317,276,112,314,216,351
3,Yoon_Park.png,"[-18.531435, 2.268928, -6.4732237, -1.7551794,...",30.235329,32.937115,33.250229,35.058430,35.874107,379,242,383,95,152
4,Joo_Won.png,"[-2.5595207, 15.16615, -0.14534593, -0.7223856...",22.880953,25.406965,27.186726,27.307686,27.335983,199,243,567,535,449


In [25]:
bestmatch = []

for i in range(0,576):
    bestmatch.append(df2.iloc[df2['1stMatch'][i], 0])
    

In [26]:
secondmatch = []

for i in range(0,576):
    secondmatch.append(df2.iloc[df2['2ndMatch'][i], 0])

In [27]:
thirdmatch = []

for i in range(0,576):
    thirdmatch.append(df2.iloc[df2['3rdMatch'][i], 0])

In [28]:
fourthmatch = []

for i in range(0,576):
    fourthmatch.append(df2.iloc[df2['4thMatch'][i], 0])

In [29]:
fifthmatch = []

for i in range(0,576):
    fifthmatch.append(df2.iloc[df2['5thMatch'][i], 0])

In [30]:
df3 = pd.DataFrame(bestmatch,columns=['Best_Match_Actor'])
df4 = pd.DataFrame(secondmatch,columns=['Second_Match_Actor'])
df5 = pd.DataFrame(thirdmatch,columns=['Third_Match_Actor'])
df6 = pd.DataFrame(fourthmatch,columns=['Fourth_Match_Actor'])
df7 = pd.DataFrame(fifthmatch,columns=['Fifth_Match_Actor'])

In [31]:
df8 = pd.concat([df2, df3, df4, df5, df6, df7], axis = 1)

In [32]:
df8.head()

,Actor,Array,Least_Array_Diff,2ndLeast_Array_Diff,3rdLeast_Array_Diff,4thLeast_Array_Diff,5thLeast_Array_Diff,1stMatch,2ndMatch,3rdMatch,4thMatch,5thMatch,Best_Match_Actor,Second_Match_Actor,Third_Match_Actor,Fourth_Match_Actor,Fifth_Match_Actor
0,Park_Hae_Soo.png,"[-4.117684, -9.592239, -7.871775, 1.1058365, 5...",30.715021,30.962044,33.221836,35.256233,35.511070,73,399,175,488,406,Yeon_Jung_Hoon.png,Joo_Jin_Mo.png,Ahn_Nae_Sang.png,Ji_Hyun_Woo.png,Shin_Hyun_Soo.png
1,Lee_Jin_Wook.png,"[65.89964, 15.4892235, -10.712205, -14.894704,...",39.949093,44.084579,47.698715,48.780590,49.159027,519,390,365,141,37,Kim_Sang_Joong.png,Lee_Bum_Soo.png,Kim_Myung_Min.png,Hong_Kyung_Min.png,Lee_Ji_Hoon.png
2,Kang_Min_Hyuk.png,"[-12.266893, -7.2090816, 1.4790905, 9.8934355,...",26.389797,30.042631,30.068354,31.712767,31.934317,276,112,314,216,351,Lee_Jung_Jin.png,Lee_Jun_Ho.png,Yoo_Yeon_Seok.png,Kim_Jung_Hyun.png,Lee_Seung_Min.png
3,Yoon_Park.png,"[-18.531435, 2.268928, -6.4732237, -1.7551794,...",30.235329,32.937115,33.250229,35.058430,35.874107,379,242,383,95,152,Ohn_Joo_Wan.png,Hoya.png,Choi_Byung_Mo.png,Hong_Jong_Hyun.png,Lee_Gi_Kwang.png
4,Joo_Won.png,"[-2.5595207, 15.16615, -0.14534593, -0.7223856...",22.880953,25.406965,27.186726,27.307686,27.335983,199,243,567,535,449,Kim_Seon_Ho.png,Lee_Dong_Wook.png,Kwon_Yool.png,Yoon_Shi_Yoon.png,Noh_Jong_Hyun.png


In [33]:
path = r"/Users/valencialie/Desktop/CZ1016_DS2/kdrama/Images/Female"
# change the working directory to the path where the images are located
os.chdir(path)

In [34]:
actresses = []

In [35]:
# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.png'):
          # adds only the image files to the flowers list
            actresses.append(file.name)

In [36]:
len(actresses)

474

In [37]:
cd

/Users/valencialie


In [38]:
cd Desktop/CZ1016_DS2/kdrama/Images/Female

/Users/valencialie/Desktop/CZ1016_DS2/kdrama/Images/Female


In [39]:
dataF = {}

In [40]:
# loop through each image in the dataset
for actress in actresses:
    # try to extract the features and update the dictionary
    try:
        featF = extract_features(actress,model)
        dataF[actress] = featF
    except:
        pass
          
# get a list of the filenames
filenamesF = np.array(list(dataF.keys()))

# get a list of just the features
featF = np.array(list(dataF.values()))
featF.shape

featF = featF.reshape(-1,4096)
featF.shape

(474, 4096)

In [41]:
pca = PCA(n_components=100, random_state=22)
pca.fit(featF)
xF = pca.transform(featF)

In [42]:
len(xF)

474

In [43]:
array_listF = list(xF)

In [44]:
array_labelF = list(dataF.keys())
F = { label: value for label, value in zip(array_labelF, array_listF) }

In [45]:
dF = pd.DataFrame(F.items(), columns=['Actress', 'Array'])

In [46]:
dF.head()

,Actress,Array
0,Jeon_Hye_Jin.png,"[10.057037, -5.9373493, 0.34505993, -12.092998..."
1,Kim_Hyun_Soo.png,"[2.198409, -10.022728, 0.036321774, 4.9317007,..."
2,Im_Soo_Jung.png,"[-2.597886, -12.319992, 9.091196, -1.5792747, ..."
3,Bae_Hae_Seon.png,"[24.630611, 0.7530353, -2.7690136, -22.539837,..."
4,Choi_Kang_Hee.png,"[10.8650255, -0.5788472, 13.763874, 1.0320615,..."


In [47]:
dist_list = []
dist_list2 = []
index = []
zipped_lists = []
sorted_zipped_lists = []
top5 = []
overallF = []

In [48]:
for j in range(0,474):
    for i in range(0,474):
        if i == j:
            continue
        else:
            dist = np.linalg.norm(xF[j]-xF[i])
            dist_list.append(dist)
            dist_list2.append(dist)
            index.append(i)
            dist_list.sort()
            top5 = dist_list[:5]
            zipped_lists = zip(dist_list2, index)
            sorted_zipped_lists = sorted(zipped_lists)
            sorted_list1 = [element for _, element in sorted_zipped_lists]
            top5.extend(sorted_list1[:5])
    overallF.append(top5)
    dist_list = []
    dist_list2 = []
    index = []
    zipped_lists = []
    sorted_list1 =[]
    top5 = []

In [49]:
dF1 = pd.DataFrame(overallF,columns=['Least_Array_Diff', '2ndLeast_Array_Diff','3rdLeast_Array_Diff', '4thLeast_Array_Diff', '5thLeast_Array_Diff', '1stMatch', '2ndMatch', '3rdMatch', '4thMatch', '5thMatch'])

In [50]:
dF2 = pd.concat([dF, dF1], axis = 1)

In [51]:
dF2.head()

,Actress,Array,Least_Array_Diff,2ndLeast_Array_Diff,3rdLeast_Array_Diff,4thLeast_Array_Diff,5thLeast_Array_Diff,1stMatch,2ndMatch,3rdMatch,4thMatch,5thMatch
0,Jeon_Hye_Jin.png,"[10.057037, -5.9373493, 0.34505993, -12.092998...",27.373398,27.758600,28.542065,28.774691,28.893175,349,47,413,376,275
1,Kim_Hyun_Soo.png,"[2.198409, -10.022728, 0.036321774, 4.9317007,...",17.859856,19.096994,20.494192,21.847498,23.016798,47,63,413,100,249
2,Im_Soo_Jung.png,"[-2.597886, -12.319992, 9.091196, -1.5792747, ...",27.188494,29.429840,29.550230,29.815596,30.265022,115,23,245,230,29
3,Bae_Hae_Seon.png,"[24.630611, 0.7530353, -2.7690136, -22.539837,...",28.366072,34.246185,37.451447,37.535973,37.550652,39,28,0,102,54
4,Choi_Kang_Hee.png,"[10.8650255, -0.5788472, 13.763874, 1.0320615,...",29.160507,31.255350,33.890415,34.438023,34.561924,235,352,88,167,140


In [52]:
bestmatchF = []

for i in range(0,474):
    bestmatchF.append(dF2.iloc[dF2['1stMatch'][i], 0])

In [53]:
secondmatchF = []

for i in range(0,474):
    secondmatchF.append(dF2.iloc[dF2['2ndMatch'][i], 0])

In [54]:
thirdmatchF = []

for i in range(0,474):
    thirdmatchF.append(dF2.iloc[dF2['3rdMatch'][i], 0])
    
fourthmatchF = []

for i in range(0,474):
    fourthmatchF.append(dF2.iloc[dF2['4thMatch'][i], 0])
    
fifthmatchF = []

for i in range(0,474):
    fifthmatchF.append(dF2.iloc[dF2['5thMatch'][i], 0])

In [55]:
dF3 = pd.DataFrame(bestmatchF,columns=['Best_Match_Actress'])
dF4 = pd.DataFrame(secondmatchF,columns=['Second_Match_Actress'])
dF5 = pd.DataFrame(thirdmatchF,columns=['Third_Match_Actress'])
dF6 = pd.DataFrame(fourthmatchF,columns=['Fourth_Match_Actress'])
dF7 = pd.DataFrame(fifthmatchF,columns=['Fifth_Match_Actress'])

In [56]:
dF8 = pd.concat([dF2, dF3, dF4, dF5, dF6, dF7], axis = 1)

In [57]:
dF8.head()

,Actress,Array,Least_Array_Diff,2ndLeast_Array_Diff,3rdLeast_Array_Diff,4thLeast_Array_Diff,5thLeast_Array_Diff,1stMatch,2ndMatch,3rdMatch,4thMatch,5thMatch,Best_Match_Actress,Second_Match_Actress,Third_Match_Actress,Fourth_Match_Actress,Fifth_Match_Actress
0,Jeon_Hye_Jin.png,"[10.057037, -5.9373493, 0.34505993, -12.092998...",27.373398,27.758600,28.542065,28.774691,28.893175,349,47,413,376,275,Lee_Elijah.png,Jung_Da_Bin.png,Han_Ye_Seul.png,Jeon_Hye_Bin.png,Kim_Hye_Yoon.png
1,Kim_Hyun_Soo.png,"[2.198409, -10.022728, 0.036321774, 4.9317007,...",17.859856,19.096994,20.494192,21.847498,23.016798,47,63,413,100,249,Jung_Da_Bin.png,Go_Won_Hee.png,Han_Ye_Seul.png,Jung_Ji_Yeon.png,Joo_Min_Kyung.png
2,Im_Soo_Jung.png,"[-2.597886, -12.319992, 9.091196, -1.5792747, ...",27.188494,29.429840,29.550230,29.815596,30.265022,115,23,245,230,29,Han__Sun_Hwa.png,Kim_Sa_Rang.png,Shin_Rin_Ah.png,Jang_Hee_Ryung.png,Park_Ha_Sun.png
3,Bae_Hae_Seon.png,"[24.630611, 0.7530353, -2.7690136, -22.539837,...",28.366072,34.246185,37.451447,37.535973,37.550652,39,28,0,102,54,Go_Doo_Shim.png,Kim_Hye_Eun.png,Jeon_Hye_Jin.png,Kim_Young_Ok.png,Lee_Yeol_Eum.png
4,Choi_Kang_Hee.png,"[10.8650255, -0.5788472, 13.763874, 1.0320615,...",29.160507,31.255350,33.890415,34.438023,34.561924,235,352,88,167,140,Lee_Bo_Young.png,Jung_Ae_Yun.png,Yeom_Jung_Ah.png,Yeom_Hye_Ran.png,Jo_Hye_Joo.png


In [211]:
#dF8.to_csv("actress.csv")

In [213]:
#df8.to_csv("actor.csv")